In [88]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


### 1. Instructions:

In order to run the code properly, please follow this recommended procedure:

1)	Click on "Cell" and then click on "Run All".

2)	Select 'Test Type' parameter (i.e., Cyclic or Monotonic). The Available Test Cases corresponding to this Test Type will be shown on the DropDown Manu below. The description details of these available test cases will be shown in the text area to the right of the test case selection area. 

3) Select 'Stage' parameter (i.e., Consolidation or Shear). 

4) Select 'Test Case(s)' and start plotting. Click on a particular test case (e.g., 'SW1') to start plotting. To plot **multiple** Test Cases simultaneously, strike **'Ctrl'** key (or **'Command'** key for Mac) and hold, then start selecting multiple test cases. As one add or drop test cases, the plot area will be dynamically updated. 

5) Finally, one can export the produced figures into PDF. The file name needs to be specified by the user by typing in the 'Fig. Name' text box and end the name with '.pdf'. 

Note: DSS = 'Direct Simple Shear'

In [89]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [90]:
%matplotlib notebook
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import numpy as np
import pandas as pd
from ipywidgets import *
from IPython.display import display
from IPython.display import clear_output
from matplotlib import rc
from matplotlib import ticker
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from scipy.signal import find_peaks_cwt
from scipy import optimize
from scipy.optimize import leastsq
import os

font = {'color':  'black',
        'weight': 'normal',
        'size': 12}

matplotlib.rcParams.update({'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
matplotlib.rcParams['axes.linewidth'] = 1.5
formatter = ticker.ScalarFormatter(useMathText=True)

In [91]:
# Init Widget Elements 

mix_type = ['SBFW','SKFW','SBSW']

select_mix_type = widgets.RadioButtons(options=mix_type, 
                                       description='Soil Mixture Type (Event)', 
                                       variable=[True, False],
                                       layout = Layout(width='25%')) 

radio_btn_test_type = widgets.RadioButtons(options=['Cyclic','Monotonic'],
                                            description='DSS Test Type',
                                            variable=[True,False],
                                            layout = Layout(width='25%'))

slider_param = FloatSlider(min=1, max=25, step=0.5, 
                           description = "Strain Level(%)", 
                           continuous_update = False)

slider_param.value = 3

layout_test = widgets.Layout(border = '0px', 
                             width='100%', 
                             height='200px', 
                             flex_flow='column', 
                             display='inline-flex')

files = os.listdir('./DSS_Dataset/')
test = [f.strip('.txt') for f in files if ('MON' in f) and ('_C.' in f) and f.startswith('SBSW')]

select_multi_test = widgets.SelectMultiple(description="Tests ID", 
                                            options= test, 
                                            disabled=False, 
                                            layout=layout_test, 
                                            justify_content='space-between')

ta = Textarea(value= "", disabled=True,
              layout=Layout(display='flex',
                            flex_flow='row', 
                            border='solid 1px',
                            align_items='stretch',
                            width='90%', 
                            height='100%'),
              description="Details of Test Cases")

btn_save_figure = widgets.Button(description='Save Figure')

In [92]:
###### Init input parameters #########
# All params defined here are global variables and are assumed to be visible throughout the notebook 
file_description = 'Description.csv'
#############################################################################################
info = pd.read_csv(file_description,delimiter=',',dtype='str',skiprows=13)
info = info.set_index('HEADING')
dssg = info.loc[:'GROUP']
dsst = info.loc['HEADING':]
dsst_col = dsst.loc['HEADING'].tolist()
dsst.columns = dsst_col
dsst = dsst[1:]
#############################################################################################

In [93]:
def on_button_save_fig_clicked(b):
    '''
    callback function for Saving Output Figures
    '''
    png = matplotlib.backends.backend_agg.Figure
    label = str(select_multi_test.value)
    
    lab1 = label+'shear1'
    lab2 = label+'shear2'
    png.savefig(b.fig_shear1, lab1, dpi=300)
    png.savefig(b.fig_shear2, lab2, dpi=300)
        
    png.clear()
    print('Figure Exported.')
    
# add action to be performed on click
btn_save_figure.on_click(on_button_save_fig_clicked)

In [94]:
def get_descriptions(dssg, test, test_type):
    '''
    Create description string based on chosen test cases and test types
    
    Input Parameters:
    test: a list of test cases strings (e.g., ["FW1", "FW2"])
    test_type: Monotonic or Cyclic
    
    Current fescription format for each test case is: 
    SW3: Salt water mix of 90% silt and 10% bentonite clay
    
    
    dependency: require 'dssg'
    '''
    
    if test_type == "Cyclic":
        descriptions = ""
    elif test_type == "Monotonic":
        descriptions = ""
        for test_case in test: 
            this_row = dssg[dssg['SPEC_REF']==test_case]

            if len(this_row) != 0:
                new_description_to_append =  test_case + ": " + \
                    str(this_row["DSSG_DESC"].values[0]) + "\n"
            else:
                new_description_to_append = test_case + ": None" + "\n"
            descriptions = descriptions + new_description_to_append            
    return descriptions

############# get detailed descriptions ##################################
def describe(test_type, test_name, description_g, description_t, IVRt):
    if (description_t['DSST_IVS'].tolist() == []):
        print("No description yet.")
        return 

    if test_type != "Cyclic":
        print(test_name +' DESCRIPTION: \n'+ description_g['DSSG_DESC'].tolist()[0])
        print('And '+description_t['DSST_DESC'].tolist()[0])
        print('Plastic Limit(PL) = ' + description_g['DSSG_PL'].tolist()[0]+
            ';\nLiquid Limit(LL) = ' + description_g['DSSG_LL'].tolist()[0]+
            ';\nPlastic Index(PI) = ' + description_g['DSSG_PI'].tolist()[0]+
            ';\nMax. past pressure Sigma_p = '+ description_g['DSSG_SIGP'].tolist()[0]+'kPa;'
             )
        if IVRt == 0.9:
            print('Initial Void Ratio assumed to be 0.9;\n')
        else:
            print('\n')
    else:
        print('No description yet.\n')
 

In [95]:
import statsmodels.api as sm
def fit_csl(sigma_v, e_c, axes):
    '''
    x is in log(sigma_v), note: log is ln here
    y is e_c
    fit a linear regression line
    
    y =  Gamma + Lambda * ln(x)
    '''
    # transform to log(sigma_v)
    logx = np.log(sigma_v)
    X = sigma_v
    y = e_c #dependent variable
    
    # y = a  + b * x
    logx = sm.add_constant(logx)
    
    # construct a model, which is an ordinary least square model (OLS)
    model = sm.OLS(y,logx)
    results = model.fit()   
    
    Gamma = results.params[0]
    Lambda = results.params[1]
    
    
    x_new = np.linspace(np.min(logx), np.max(logx*1.2),100)
    y_new = Gamma + Lambda * x_new
    #axes.semilogx(x_new, powerlaw(x_new, a, b))
    e_logx = np.exp(x_new)

    axes.semilogx(e_logx, y_new)
    
    return (Gamma, Lambda, results.rsquared)

In [96]:
def fit_powerlaw(x, y, axes):
    '''
    Given x and y, this function solves for S and m.
    '''
    yerr = [i * 0.2 for i in y]
    logx = np.log10(x)
    logy = np.log10(y)

    yerr = np.asarray(yerr)
    y = np.asarray(y)
    x = np.asarray(x)
    logyerr = yerr / y

    fitfunc = lambda p, x: p[0] + p[1] * x
    errfunc = lambda p, x, y, err: (y - fitfunc(p, x)) / err

    pinit = [1.0, -1.0]
    out = optimize.leastsq(errfunc, pinit,
                           args=(logx, logy, logyerr), full_output=1)

    pfinal = out[0]
    covar = out[1]
    b = pfinal[1]
    a = 10.0**pfinal[0]

    powerlaw = lambda x, a, b: a * (x**b)
    
    x_new = np.linspace(np.min(x*0.8), np.max(x*1.2),100)
    axes.semilogx(x_new, powerlaw(x_new, a, b))
    
    return (a,b)

In [97]:
def plot_CSR(time, shear_strain, ax, peak_avg, period, initial_vertical_stress,label_CSR):
    
    time_def_percent = time[-1]
    CSR = round(float(peak_avg/initial_vertical_stress),4)
    Number_of_Cycle = round(float(time_def_percent/period),1)
                
    ax.semilogx(Number_of_Cycle, CSR, 'o', label = label_CSR)
    ax.set_xlabel('Number of Cycle, N', fontdict=font)
    ax.set_ylabel('Cyclic Stress Ratio, $\\tau_{cyc}/\sigma\'_{vc}$', fontdict=font)
    ax.set_title('CSR vs. N', fontdict=font)
    ax.set_xlim([0.1,100])
    fmt = FuncFormatter(lambda x, pos: str(int(x)))
    ax.xaxis.set_major_formatter(fmt)
    ax.grid(True, which='both')
    ax.legend(bbox_to_anchor=(1, 1),prop={'size':10}) 
    
    print (label_CSR + '; CSR ='+ str(CSR) + '; Number of Cycle: '+str(Number_of_Cycle))
    return (Number_of_Cycle, CSR)

In [98]:
def strength_normalize(OCR, initial_vertical_stress, shear_strain, shear_stress, ax, strain_limit):

    diff_arr = abs(abs(shear_strain) - strain_limit)
    index_min = diff_arr.argmin()
    residual = shear_stress[index_min]
    Su_sigma = residual/initial_vertical_stress

    ax.semilogx(OCR,Su_sigma,'o')
    fmt = FuncFormatter(lambda x, pos: str(int(x)))
    ax.xaxis.set_minor_formatter(fmt)
    ax.xaxis.set_major_formatter(fmt)
    ax.set_title('Strength Normalization Plot')
    ax.set_ylabel(r'$\mathrm{\frac{S_u}{\sigma_v}}$', fontsize=14, rotation=360)
    ax.set_xlim([1,10])
    ax.set_xlabel('OCR')
    ax.grid(True, which="both")
    
    return(OCR,Su_sigma)
  

In [99]:
def plot_all(test_type, select_multi_test):
     
    test_chose = list(select_multi_test)
    L = len(test_chose)
    
    # for plotting later 
    Number_of_Cycle_arr = np.zeros(L)*np.nan 
    CSR_arr = np.zeros(L)*np.nan
    Void_Ratio_arrf = np.zeros(L)*np.nan
    Vertical_stress_arrf = np.zeros(L)*np.nan
    OCR_arr = np.zeros(L)*np.nan
    Su_arr = np.zeros(L)*np.nan
            
    if L > 0:
        f, axes = plt.subplots(1,2,figsize=(8,4))
        btn_save_figure.fig_shear = f

        for i in range(0,L):
            path = './DSS_Dataset/'+test_chose[i] + '.txt'
            data = np.genfromtxt(path, delimiter=',')

            time_raw = data[2: ,0] - data[2,0]
            shear_stress = data[2: ,17]
            shear_strain = data[2: ,14] 
            
            #define plotting range according to strain limit value
            strain_limit = slider_param.value
            is_abs_above_limit = abs(shear_strain) > strain_limit
            index_limit = np.where(is_abs_above_limit)[0][0]
            index_min2 = index_limit+1

            if test_type == 'Cyclic':
                lab = test_chose[i]
                driving = data[2:,22]
                start_index = np.where(driving>0)[0][0]
                driving = driving[start_index:index_min2]
                
                pwp = data[start_index:index_min2,20]
                vertical_stress = data[start_index:index_min2,18]
                initial_vertical_stress = vertical_stress[0]
                Ru = pwp/initial_vertical_stress
                void_ratio = 0*time_raw
                
                time = time_raw[start_index:index_min2]-time_raw[start_index]
                shear_strain = shear_strain[start_index:index_min2]
                shear_stress = shear_stress[start_index:index_min2]

                for n in range(1,len(driving)):
                    if (driving[n]>driving[n-1]) & (driving[n]>driving[n+1]):
                        first_peak=n
                        peak1=shear_stress[first_peak]
                        peak2=shear_stress[first_peak*3]
                        period = (time[n+1])*4
                        break
                peak_avg =(abs(peak1)+abs(peak2))/2
                cycle_all = time/period

                lab_csr = lab +'; strain:'+ str(slider_param.value)+'%'
                (Number_of_Cycle, CSR) = plot_CSR(time, shear_strain,axes[0], peak_avg, period, 
                                                initial_vertical_stress, lab_csr)
                f.tight_layout()
                Number_of_Cycle_arr[i] =Number_of_Cycle
                CSR_arr[i] =CSR
                axes[1].axis('off')
                
            else:
                shear_strain = data[2:index_min2,14]
                time = data[2:index_min2,0] - data[2,0]
                pwp = data[2:index_min2,20]
                shear_stress = data[2:index_min2,17]
                vertical_stress = data[2:index_min2,18]
                
                FVSt = vertical_stress[-1]

                initial_vertical_stress = vertical_stress[0] 
                
                condition = (dsst['SPEC_REF']==test_chose[i])&(dsst['DSST_FSET']=='Shear.txt')
                description_g = dssg[(dssg['SPEC_REF']==test_chose[i])]
                description_t = dsst[condition]
                
                #initial and final void ratio:
                try:
                    IVRt = float(description_t['DSST_IVR'].tolist()[0])
                    FVRt = float(description_t['DSST_FVR'].tolist()[0])
                    OCR = str(int(description_t['DSST_IOCR'].tolist()[0]))        
                    lab = test_chose[i] + '; OCR = '+ OCR +'; strain:'+ str(slider_param.value)+'%'

                except IndexError:
                    IVRt = 0.9
                    FVRt = 0.8        
                    OCR = 1
                    lab = test_chose[i] + '; OCR = 1; strain:'+ str(slider_param.value)+'%'

                
                describe(test_type=test_type, test_name=test_chose[i], 
                         description_g=description_g, description_t=description_t, 
                         IVRt=IVRt)    
                
                Void_Ratio_arrf[i] = FVRt
                Vertical_stress_arrf[i] = FVSt
                
                axes[0].semilogx(FVSt, FVRt, '^', label=test_chose[i]+'final')
                axes[0].set_ylim([FVRt-0.1,IVRt+0.1])
                axes[0].set_xlabel('Vertical Stress, $\sigma\'_{vc}$ (kPa)', fontdict=font)
                axes[0].set_ylabel('Void Ratio, e', fontdict=font)
                axes[0].set_title('Critical State Line', fontdict=font)
                axes[0].legend(bbox_to_anchor=(1, 1),prop={'size':8})
                axes[0].grid(True,which='both')
                
                (OCR, Su_sigma) = strength_normalize(OCR, initial_vertical_stress, shear_strain, shear_stress, 
                                                     axes[1], strain_limit)
                
                OCR_arr[i] =OCR
                Su_arr[i] =Su_sigma

                
    if L >= 2 and test_type == "Cyclic":
        (a,b)=fit_powerlaw(Number_of_Cycle_arr, CSR_arr, axes[0])
        a = round(float(a),3)
        b = round(float(b),3)
        axes[0].text(0.5, 0.5,r'${CRR = %.2f*N^{%.2f}}$' % (a,b), fontsize=12,
              transform=axes[0].transAxes)  
        
    elif L >= 2 and test_type == "Monotonic":
        ################ x=vertical_stress; y=void_ratio ####################
        (gamma, labda, rsquared) = fit_csl(Vertical_stress_arrf, Void_Ratio_arrf, axes[0])
        axes[0].text(0.25,0.5,r'$CSL: e_c=%.2f%.2f*log(\sigma_v)$' %(gamma, labda),fontsize=12, 
                    transform=axes[0].transAxes)
        
        (S,m)=fit_powerlaw(OCR_arr, Su_arr, axes[1])
        axes[1].text(0.5,0.5,r'$\frac{S_u}{\sigma_v}=%4.2f * OCR^{%4.2f}$' %(S, m),fontsize=12, 
                     horizontalalignment='center',verticalalignment='center', transform=axes[1].transAxes)   

In [103]:
def on_options_change(*args):

    files = os.listdir('./DSS_Dataset/')
    test_type = radio_btn_test_type.value
    mix_type = select_mix_type.value
    
    # if new mix types are added, should update accordingly
    if mix_type == 'SBSW':
        files = [f for f in files if f.startswith('SBSW')]
    if mix_type == 'SBFW':
        files = [f for f in files if f.startswith('SBFW')]
    if mix_type == 'SKFW':
        files = [f for f in files if f.startswith('SKFW')]   

    if test_type == 'Cyclic':
        test1 = [f for f in files if 'CYC' in f]
        test = [f.strip('.txt') for f in test1 if '_S.' in f]
    if test_type == 'Monotonic':
        test1 = [f for f in files if 'MON' in f]  
        test = [f.strip('.txt') for f in test1 if '_S.' in f]     
    
    select_multi_test.options = test
    
    ##### Update Text Area ########
    ta.value = get_descriptions(dssg = dssg, 
                                test = select_multi_test.options, 
                                test_type= radio_btn_test_type.value)
    
    
def update_plot(*args):
    clear_output()
    plot_all(test_type = radio_btn_test_type.value, select_multi_test = select_multi_test.value)
    
def on_slider_change(*args):
    if (radio_btn_test_type.value == "Cyclic"):        
        update_plot(*args)
    if (radio_btn_test_type.value == "Monotonic"):  
        update_plot(*args)

        
# add test_type observer
radio_btn_test_type.observe(on_options_change, names="value")
select_mix_type.observe(on_options_change, names="value")
select_multi_test.observe(update_plot, names="value")

# update plot when slider changes and filter is matched 
slider_param.observe(on_slider_change, names="value")

hbox_param_major = widgets.HBox([select_mix_type, radio_btn_test_type, slider_param], 
                                layout = Layout(display='flex',
                                                flex_flow='row', 
                                                border='solid 1px',
                                                align_items='stretch', 
                                                width='100%',
                                                height="100px"))

vbox_test = widgets.VBox([select_multi_test], 
                         layout = Layout(overflow_y = "scroll", 
                                         display='flex',
                                         flex_flow='row', 
                                         border='solid 1px',
                                         align_items='stretch', 
                                         width="40%")) 

hbox_main =widgets.HBox([vbox_test, ta],
                        layout = Layout(display='flex',
                                        flex_flow='row', 
                                        border='solid 1px',
                                        align_items='stretch', 
                                        width='100%',
                                        height="60%"))

hbox_save_fig = widgets.HBox([btn_save_figure],
                             layout = Layout(display='flex',
                                             flex_flow='row',
                                             align_items='center', 
                                             width='100%',
                                             height="50px"))      

display(hbox_param_major)
display(hbox_main)
display(hbox_save_fig)